# Predicting output sequence in Multi-Stage Continuous Process

### This problem refer the dataset obtained from kaggle (Provided by Liveline)


https://www.kaggle.com/supergus/multistage-continuousflow-manufacturing-process

##  Tasks achieved:

### The initial goal over here is to predict the output sequence 
     
     1)  There are about 28 characteristics that needs to be predicted
     2)  The ML model should determine what characteristics can be predicted based on the input sequence
     3)  For convenience used Auto-ML library TPOT  that helps in hyper-parameter tuning the model for 
         each characteristic more information about tpot can be found at 
     4)  Currently, a regression based approach is used to predict output sequence
     5)  After predicting the output signal, the actual and predicted outputs are compared with the warning limit                        setting to identify the good and bad signals
     6)  Accuracy metric MAE has been used to identify the optimum model 
     7)  The model files and normalization files are stored as joblib file that helps in deployment
     8)  The feature importance plot helps to identify what characteristic could help in predicting the sequence
     9)  The accuracy metric r2 helps to explain how much variability can be explained by the sequence
     10) The residual plot helps to understand which models are working well


##   More to Come:
      
      1) Output needs to be discussed with clients to see if it makes sense or any logic needs to be added
      2) This problem needs to be converted to time-series format to predict the upcoming  sequence 
      3) A classification model could be developed to predict if a input sequence is good or bad
      4) As of now, the extreme outliers are removed from consideration. But if our focus is on outliers, anomaly detection   
         models could be applied (including Auto-Encoders, Isolation Forest)
      5) A flask based app can be deployed in Heroku (or) any private cloud that helps to share results with customers and for
         deploying models in real-time
      6) The ML model provided here can be used as a baseline . A Deep learning based (LSTM, CNN) based approach needs to be 
         tested to see if accuracy can be improved further
      7) Need to include some message brokers strategies to deploy a real-time solution

### Declare Dependencies
    Identify the output fields and the threshold limits

In [ ]:
import pandas as pd
import dask  # For Parallel processing
from tpot import TPOTRegressor # Auto-ML
from sklearn.model_selection import train_test_split, KFold # Stratified train, test split and cross validation
import os # To set path
from sklearn.preprocessing import StandardScaler  # Normalization techniques
from sklearn.metrics import mean_squared_error,r2_score # Accuracy metrics
from joblib import dump, load  # Loading and dumping model files
import matplotlib.backends.backend_pdf  # Adding plots to pdf
import matplotlib.pyplot as plt # Creating Plot
import seaborn as sns # Creating Plot
import math # For Calculating rmse
import warnings # For ignoring warnings from pandas
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.read_csv("continuous_factory_process.csv")

output_col = df.filter(regex = 'Output').columns.copy()
setpoint_col = df.filter(regex = 'Setpoint').columns.copy()

output_col= sorted(set(output_col) - set(setpoint_col))

### Generate a variable called Setting based on RawMaterial Properties to stratify train test split

In [ ]:
raw_material_properties = list(df.filter(regex = 'RawMaterial.Property').columns.copy())
unique_comb_parameters = df.groupby(raw_material_properties).size().reset_index().rename(columns={0:'count'})
unique_comb_parameters.index.name = 'Setting'
unique_comb_parameters.drop('count', axis=1, inplace=True)
unique_comb_parameters = unique_comb_parameters.reset_index()
unique_comb_parameters['Setting'] = 'S'+ unique_comb_parameters['Setting'].astype(str)
#df.groupby(raw_material_properties)

### Merge the setting variable with the dataframe and identify the input columns for training

In [ ]:
df = pd.merge(df, unique_comb_parameters, on = raw_material_properties, how = 'left')
input_col = df.columns.drop(df.filter(regex='Output|Setpoint|time_stamp').columns).copy()
input_col = list(input_col)

### Perform Pre-Processing, Train & test model, store model files and test effectiveness of the model

In [ ]:
#filt_df = df[input_col]

winning_pipes = []
scores = []
cross_val_scores = pd.DataFrame(columns = ['CHARACTERISTIC_NAME', 'R2_score', 'MAE', 'RMSE'])
selected_features = pd.DataFrame(columns = ['CHARACTERISTIC_NAME', 'INPUT_FEATURES'])


# Define the directory location here ex: C:/Test/Feature_Imp_Plot.pdf
feat_imp_plot = matplotlib.backends.backend_pdf.PdfPages("Feature_Imp_Plot_v2.pdf")
line_plot = matplotlib.backends.backend_pdf.PdfPages("Line_Plot_v2.pdf")
resid_plot = matplotlib.backends.backend_pdf.PdfPages("resid_plot_v2.pdf")


# Predict the output sequence for each sequence
for col in output_col:
    overall_col = input_col.copy()
    overall_col.append(col)
    filt_df = df[overall_col]
    
    filt_df = filt_df.loc[filt_df[col]>0].copy() # many records with 0 as output value are removed
    spec_limit = filt_df.groupby(['Setting']).agg({col:{'Mean':'mean', 'Std':'std'}})
    spec_limit.columns = spec_limit.columns.droplevel(0) 
    spec_limit = spec_limit.reset_index()
    
    # For removing outliers +/- 3sigma limits are used to remove extreme outliers for each setting
    new_df = pd.merge(filt_df, spec_limit, on = ['Setting'], how = 'inner')
    new_df  = new_df.loc[(new_df[col] > new_df['Mean'] - 3* new_df['Std'])
                             & (new_df[col] < new_df['Mean'] + 3* new_df['Std'])].copy()

    new_df = new_df.drop(['Mean', 'Std'], axis=1)
    
    
    # Filter Input Factors based on corr value with output
    corr_matrix = new_df.corr().abs()
    corr_matrix = corr_matrix.loc[corr_matrix.index == col].copy()
    filt_col = [c for c in corr_matrix.columns if any(corr_matrix[c] > 0.3)] # Filter columns with higher correlation
    
    
    # Split the dataset into training and testinng stratified based on setting parameter
    stratified_col = filt_col.copy()
    stratified_col.append('Setting')
    new_df = new_df[stratified_col]
    new_df = new_df.dropna()
    
    filt_df = new_df.loc[new_df[col]>1].copy()
    
    if (isinstance(filt_df, pd.DataFrame)):
        if(len(filt_df)>1):
            y = new_df['Setting']
            # Stratify training and testing
            train_x, test_x, train_y, test_y = train_test_split(new_df, new_df[col], test_size = 0.25,stratify = y, shuffle = True)
            train_x = train_x.drop(['Setting', col], axis = 1)
            test_x = test_x.drop(['Setting', col], axis = 1)
            
            # Apply normalization techniques to perform standardization
            sc = StandardScaler()
            train_x_trans = sc.fit_transform(train_x)
            test_x_trans = sc.transform(test_x)
    
            #print ("Number of Records = ", len(train_x))
            
            
            cv = KFold(n_splits=3)
            cv_iter = list(cv.split(train_x_trans, train_y))
            tpot = TPOTRegressor(verbosity = 1, random_state = 15, n_jobs = -1, generations = 10, population_size = 50, 
                                 memory = None,
                                 cv=cv_iter,use_dask = True)
            # Tpot will use dask to parallel process multiple model, overall 600 different models are expeted

            tpot.fit(train_x_trans, train_y)
            winning_pipes.append(tpot.fitted_pipeline_) # Best Pipeline is stored to a .py file

            scores.append(tpot.score(test_x_trans, test_y)) # Test Scores are stored
            
            
            # The selected features corresponding to each output filed are stored and also cross val scores are stored
            selected_features = selected_features.append({'CHARACTERISTIC_NAME': col, 'INPUT_FEATURES': list(train_x.columns)}, ignore_index = True)
            cross_val_scores = cross_val_scores.append({'CHARACTERISTIC_NAME': col, 'R2_score': r2_score(train_y, tpot.predict(train_x_trans)),
                                                        'MAE':mean_squared_error(test_y, tpot.predict(test_x_trans)),
                                                        'RMSE':math.sqrt(mean_squared_error(test_y, tpot.predict(test_x_trans)))
                                                       }, ignore_index=True)
            #print ("Cross Val Score = ", cross_val_scores)
            
            
            model_directory = os. getcwd() 
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            model_file = model_directory+ col+ ".py"
            norm_file = model_directory+ col+ "norm"+ ".joblib"
            model_joblib = model_directory+ col+ ".joblib"
            tpot.export(model_file)
            
            # Store the model files and norm files 
            dump(tpot.fitted_pipeline_, model_joblib)
            dump(sc,norm_file)
            
            # Accuracy Metrics
            r2_val = r2_score(test_y, tpot.predict(test_x_trans))
            mae_val = mean_squared_error(test_y, tpot.predict(test_x_trans))
            
            
            
            train_x['Predicted'] = tpot.predict(train_x_trans)
            test_x['Predicted'] = tpot.predict(test_x_trans)
            train_x['Output'] = train_y#tpot.predict(X_test)
            test_x['Output'] = test_y#tpot.predict(X_test)

            
            limit_col = col.replace('Actual', 'Setpoint')
            max_limit = df[limit_col].max()
            
            train_x['Set_Point'] = max_limit
            train_x['Actual_Warning'] = 0
            train_x['Predicted_Warning'] = 0
            train_x.loc[train_x['Output']>max_limit, 'Actual_Warning'] = 1
            train_x.loc[train_x['Predicted']>max_limit, 'Predicted_Warning'] = 1
            
            test_x['Set_Point'] = max_limit
            test_x['Actual_Warning'] = 0
            test_x['Predicted_Warning'] = 0
            test_x.loc[test_x['Output']>max_limit, 'Actual_Warning'] = 1
            test_x.loc[test_x['Predicted']>max_limit, 'Predicted_Warning'] = 1
            
            training_file = col + ".csv"
            testing_file =  col + ".csv"

            train_x.to_csv(training_file)
            test_x.to_csv(testing_file)
            
            
            fig = plt.figure(figsize = (10,5))
            extracted_best_model = tpot.fitted_pipeline_.steps[-1][1]
            extracted_best_model.fit(train_x[selected_features['INPUT_FEATURES'][0]], train_y)
            if hasattr(extracted_best_model, 'coef_'):
                feat_importances = pd.Series(extracted_best_model.coef_, index=selected_features['INPUT_FEATURES'][0]).sort_values(ascending = False)
                feat_importances.plot(kind='barh', title = col)
                plt.show()
                feat_imp_plot.savefig(fig,bbox_inches = 'tight',dpi = 100)
            if hasattr(extracted_best_model, 'feature_importances_'):
                feat_importances = pd.Series(extracted_best_model.feature_importances_, index=selected_features['INPUT_FEATURES'][0]).sort_values(ascending = False)
                feat_importances.plot(kind='barh', title = col)
                plt.show()
                feat_imp_plot.savefig(fig,bbox_inches = 'tight',dpi = 100)
            
            
            sns_plot = sns.lmplot(x='Output', y="Predicted", data=test_x)# hue = "MACHINE_NUMBER")
            fig = sns_plot.fig
            plt.suptitle(col +"\n" + "R2 Score = "+ str(round(r2_val,4)) + " , MSE =" +str(round(mae_val,4)), fontsize  = 12)
            line_plot.savefig(fig,bbox_inches = 'tight',dpi = 100)
            #(pd.Series(model.feature_importances_, index=X.columns).nlargest(4).plot(kind='barh'))

            
line_plot.close()
feat_imp_plot.close()
selected_features.to_csv("Inputfeatures_selected_v3.csv")
cross_val_scores.to_csv("Cross_Val_Score_Testing_v3.csv")
    
    